<a href="https://colab.research.google.com/github/Adrian1228/PROGCOM-A/blob/main/JUEGO_HALLOWEEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import random
import winsound

# CONFIGURACIÓN GENERAL
ANCHO = 1000
ALTO = 700
VELOCIDAD_JUGADOR = 25
VELOCIDAD_ENEMIGO = 10

class Jugador:
    def __init__(self, canvas):
        self.canvas = canvas
        self.imagen = ImageTk.PhotoImage(Image.open("D:/nino.png").resize((90, 90)))
        self.id = canvas.create_image(150, ALTO//2, image=self.imagen)

    def mover(self, event):
        x, y = self.canvas.coords(self.id)
        if event.keysym == "Up" and y > 45:
            self.canvas.move(self.id, 0, -VELOCIDAD_JUGADOR)
        elif event.keysym == "Down" and y < ALTO - 45:
            self.canvas.move(self.id, 0, VELOCIDAD_JUGADOR)
        elif event.keysym == "Left" and x > 45:
            self.canvas.move(self.id, -VELOCIDAD_JUGADOR, 0)
        elif event.keysym == "Right" and x < ANCHO - 45:
            self.canvas.move(self.id, VELOCIDAD_JUGADOR, 0)

class Enemigo:
    def __init__(self, canvas, tipo):
        self.canvas = canvas
        self.tipo = tipo
        self.imagen = ImageTk.PhotoImage(Image.open(f"D:/{tipo}.png").resize((90, 90)))
        self.id = canvas.create_image(
            random.randint(ANCHO//2, ANCHO - 60),
            random.randint(80, ALTO - 80),
            image=self.imagen
        )

    def mover(self):
        self.canvas.move(self.id, -VELOCIDAD_ENEMIGO, 0)
        if self.canvas.coords(self.id)[0] < 0:
            self.canvas.coords(self.id, ANCHO, random.randint(80, ALTO - 80))

class Dulce:
    def __init__(self, canvas):
        self.canvas = canvas
        self.imagen = ImageTk.PhotoImage(Image.open("D:/dulce.png").resize((60, 60)))
        self.id = canvas.create_image(
            random.randint(150, ANCHO - 150),
            random.randint(80, ALTO - 80),
            image=self.imagen
        )

class Juego:
    def __init__(self, root):
        self.root = root
        self.root.title("Halloween Candy Hunt")

        self.canvas = tk.Canvas(root, width=ANCHO, height=ALTO)
        self.canvas.pack()

        self.fondo_img = Image.open("D:/fondo_halloween.jpg").resize((ANCHO, ALTO))
        self.fondo = ImageTk.PhotoImage(self.fondo_img)
        self.canvas.create_image(0, 0, image=self.fondo, anchor="nw")

        # Pantalla inicial
        self.titulo = self.canvas.create_text(
            ANCHO//2, ALTO//3, text="HALLOWEEN CANDY HUNT",
            fill="orange", font=("Chiller", 60, "bold")
        )

        self.boton_play = tk.Button(root, text="PLAY", font=("Arial", 30, "bold"),
                                    bg="orange", fg="black", command=self.iniciar_juego)
        self.boton_play_window = self.canvas.create_window(ANCHO//2, ALTO//2, window=self.boton_play)

    def sonido(self, archivo):
        winsound.PlaySound(archivo, winsound.SND_ASYNC)

    def iniciar_juego(self):
        # Sonido de bienvenida
        self.sonido("D:/bienvenidos.wav")

        # Eliminar menú
        self.canvas.delete("all")
        self.boton_play.destroy()

        self.canvas.create_image(0, 0, image=self.fondo, anchor="nw")

        self.jugador = Jugador(self.canvas)
        self.dulce = Dulce(self.canvas)

        self.enemigos = [
            Enemigo(self.canvas, "fantasma"), Enemigo(self.canvas, "fantasma"),
            Enemigo(self.canvas, "zombie"), Enemigo(self.canvas, "zombie"),
            Enemigo(self.canvas, "vampiro"), Enemigo(self.canvas, "vampiro")
        ]

        self.puntaje = 0
        self.meta = 30
        self.tiempo = 60

        self.texto_puntaje = self.canvas.create_text(100, 30, text="Puntaje: 0", fill="white", font=("Arial", 20, "bold"))
        self.texto_meta = self.canvas.create_text(270, 30, text="Meta: 30", fill="yellow", font=("Arial", 20, "bold"))
        self.texto_tiempo = self.canvas.create_text(450, 30, text="Tiempo: 60s", fill="red", font=("Arial", 20, "bold"))

        self.root.bind("<KeyPress>", self.jugador.mover)
        self.jugando = True
        self.actualizar_tiempo()
        self.actualizar()

    def colision(self, id1, id2):
        x1, y1 = self.canvas.coords(id1)
        x2, y2 = self.canvas.coords(id2)
        return abs(x1 - x2) < 50 and abs(y1 - y2) < 50

    def actualizar_tiempo(self):
        if not self.jugando:
            return
        if self.tiempo > 0:
            self.tiempo -= 1
            self.canvas.itemconfig(self.texto_tiempo, text=f"Tiempo: {self.tiempo}s")
            self.root.after(1000, self.actualizar_tiempo)
        else:
            self.game_over("¡Se acabó el tiempo!")

    def actualizar(self):
        global VELOCIDAD_ENEMIGO
        if not self.jugando:
            return

        for enemigo in self.enemigos:
            enemigo.mover()
            if self.colision(self.jugador.id, enemigo.id):
                self.game_over("¡Un monstruo te atrapó!")
                return

        if self.colision(self.jugador.id, self.dulce.id):
            self.puntaje += 1
            self.meta -= 1

            self.sonido("D:/excelente.wav")  # sonido dulce

            self.canvas.itemconfig(self.texto_puntaje, text=f"Puntaje: {self.puntaje}")
            self.canvas.itemconfig(self.texto_meta, text=f"Meta: {self.meta}")
            self.canvas.coords(self.dulce.id, random.randint(150, ANCHO - 150), random.randint(80, ALTO - 80))

            if self.puntaje % 5 == 0:
                VELOCIDAD_ENEMIGO += 2

            if self.puntaje >= 30:
                self.game_over("¡GANASTE!")
                return

        self.root.after(80, self.actualizar)

    def game_over(self, mensaje):
        self.jugando = False

        # sonido de perder
        self.sonido("D:/nooo.wav")

        self.canvas.create_text(ANCHO//2, ALTO//2, text=mensaje, fill="red", font=("Arial", 50, "bold"))
        boton = tk.Button(self.root, text="Reintentar", font=("Arial", 16, "bold"), bg="orange", fg="black", command=self.reiniciar)
        self.canvas.create_window(ANCHO//2, ALTO//2 + 80, window=boton)

    def reiniciar(self):
        self.canvas.delete("all")
        self.__init__(self.root)

root = tk.Tk()
Juego(root)
root.mainloop()
